<div style="text-align: right"><a href="http://ml-school.uni-koeln.de">Virtual Summer School "Deep Learning for
    Language Analysis"</a> <br/><strong>Text Analysis with Deep Learning</strong><br/>Aug 30 — Sep 3, 2021<br/>Nils Reiter<br/><a href="mailto:nils.reiter@uni-koeln.de">nils.reiter@uni-koeln.de</a></div>

# Exercise 0: Getting Started

This is the first exercise for you to solve independently, but as a group of approximately three students. Feel free to contact us via [Teams](https://teams.microsoft.com/l/team/19%3ao0bSbRYzteerqETtEm1iTg_zkTAplnFrx48z7G_LUJ81%40thread.tacv2/conversations?groupId=384c7fc6-ad91-4f72-b71e-d0e70a909d35&tenantId=4982814a-1107-493e-ac2f-3356509a8687) and call us into your room if you need support. 

We will first verify that libraries can be imported properly.

In [1]:
import keras
import numpy as np
import pandas as pd